# 实现最基本的 Attention 机制

## 向量的点积（点乘）
> torch.dot(x,y)跟 x @ y 是一样的，
## torch.softmax()
dim = 1, 向量的第n个维度上，1 就是第二个，如果是二维的话，1 和 -1 反而是一样的，
dim = -1,向量的最后一个维度上

tensor([[ 0.5793, -1.7151, -1.4405, -0.1477],
        [-1.1456,  0.2692,  0.8947,  0.6108],
        [-0.9580, -1.0135,  1.4974,  1.2203]])

这是一个 3*4 的 tensor， shape 的话是 [3,4]


tensor([[[ 1.0994,  0.3531],
         [-1.1037, -1.0341],
         [ 1.0761,  1.0038],
         [-0.5333,  0.5775]],

        [[ 1.5154, -1.2769],
         [ 0.2073,  0.4081],
         [-1.3587,  1.0832],
         [-1.4338,  0.1262]],

        [[-0.5507, -0.6281],
         [-0.7851,  1.4680],
         [ 0.0486,  0.4189],
         [-0.8566, -0.2512]]])

这是一个 [3,4,2]的 tensor，实际打印出来是反过来的，最里面的单元是2 个浮点数组成的，4 个这样的组合，组成第二层单元，这样的单元，有3 个，
从里到外的一个过程，这一点在后续处理各种 tensor 很重要，要对 tensor 有这种基本的感觉，以后各种数据一来就知道是什么 shape，对数据的 shape 要非常敏感
整个神经网络模型，实际上并不是网络，实际是各种 shape 的转换，神经网络只是一种说法，用来糊弄糊弄外行的，实际机器学习就是玩各种 shape，跟 GPU 打交道也是 shape，参数也是 shape，根本不存在像神经一样的网络，基本不沾什么边，






In [6]:
import torch

test1 = torch.randn(6, 3)
test1




tensor([[ 0.5060,  0.3686,  1.1199],
        [-2.6047,  0.3158, -0.7354],
        [ 0.1548, -0.4813, -0.3124],
        [ 0.0210,  1.2137,  1.6285],
        [ 0.3561, -0.1072,  1.3742],
        [-1.2743,  0.8822,  0.8836]])

In [29]:
input=torch.tensor([[ 0.5060,  0.3686,  1.1199],
        [-2.6047,  0.3158, -0.7354],
        [ 0.1548, -0.4813, -0.3124],
        [ 0.0210,  1.2137,  1.6285],
        [ 0.5060,  0.3686,  1.1199],  #重复的行
        [-1.2743,  0.8822,  0.8836]])

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
input = input.to(device)

input_d = input.shape[-1]# 获取最后一个维度的大小, 代表每个 token 的维度，就是所谓的d_model
#Wq需要将 input 从 d_model 维度映射到 d_k 维度，当然是在实际应用中，都是 d_k < d_model的，比如从 7168 降为 128 之类，
d_k = 4  #假设我们要把维度映射到4维
Wq = torch.nn.Parameter(torch.randn(input_d, d_k), requires_grad=True).to(device)  #权重矩阵，维度是 [d_model, d_k]
Wk = torch.nn.Parameter(torch.randn(input_d, d_k), requires_grad=True).to(device)  #权重矩阵，维度是 [d_model, d_k]
Wv = torch.nn.Parameter(torch.randn(input_d, d_k), requires_grad=True).to(device)  #权重矩阵，维度是 [d_model, d_k]

Q = torch.matmul(input, Wq)  #计算 Q 矩阵，维度是 [seq_len, d_k]
K = torch.matmul(input, Wk)  #计算 K 矩阵，维度是 [seq_len, d_k]
V = torch.matmul(input, Wv)  #计算 V 矩阵，维度是 [seq_len, d_k]

atten_scores = torch.matmul(Q, K.T) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))  #计算注意力分数，维度是 [seq_len, seq_len]


atten_weights = torch.softmax(atten_scores, dim=-1)  #计算注意力权重，维度是 [seq_len, seq_len]


atten_scores, atten_weights


(tensor([[  0.8547,  -0.8081,  -0.3946,   1.4745,   0.8547,   0.6038],
         [ -0.9138, -11.5930,   4.8040, -10.0269,  -0.9138, -11.4487],
         [ -0.1433,   2.2329,  -0.6218,   1.0937,  -0.1433,   1.6883],
         [  1.0512,  -5.6924,   1.0857,  -1.2394,   1.0512,  -3.2838],
         [  0.8547,  -0.8081,  -0.3946,   1.4745,   0.8547,   0.6038],
         [  0.3857,  -9.8191,   3.1184,  -5.6999,   0.3857,  -7.9094]],
        device='mps:0', grad_fn=<DivBackward0>),
 tensor([[1.9558e-01, 3.7083e-02, 5.6072e-02, 3.6351e-01, 1.9558e-01, 1.5218e-01],
         [3.2655e-03, 7.5169e-08, 9.9347e-01, 3.5988e-07, 3.2655e-03, 8.6836e-08],
         [4.3339e-02, 4.6653e-01, 2.6859e-02, 1.4932e-01, 4.3339e-02, 2.7061e-01],
         [3.1741e-01, 3.7401e-04, 3.2852e-01, 3.2123e-02, 3.1741e-01, 4.1583e-03],
         [1.9558e-01, 3.7083e-02, 5.6072e-02, 3.6351e-01, 1.9558e-01, 1.5218e-01],
         [5.7550e-02, 2.1289e-06, 8.8475e-01, 1.3095e-04, 5.7550e-02, 1.4372e-05]],
        device='mps:0', g



# 测试是否在 device 上运行，运行 10000 次 看看时间
import time
start_time = time.time()
for _ in range(1000000):
    Q = torch.matmul(input, Wq)  #计算 Q 矩阵，维度是 [seq_len, d_k]
    K = torch.matmul(input, Wk)  #计算 K 矩阵，维度是 [seq_len, d_k]
    V = torch.matmul(input, Wv)  #计算 V 矩阵，维度是 [seq_len, d_k]
    atten_scores = torch.matmul(Q, K.T) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))  #计算注意力分数，维度是 [seq_len, seq_len]
    atten_weights = torch.softmax(atten_scores, dim=-1)  #计算注意力权重，维度是 [seq_len, seq_len]
    output = torch.matmul(atten_weights, V)  #计算输出，维度是 [seq_len, d_k]       
end_time = time.time()
print(f"Time taken for 1000000 iterations: {end_time - start_time} seconds")

# 现在还没有位置编码，所以经过 Attention 处理以后，有两个值还是是一样的

# 点积与矩阵乘法
## 矩阵乘法，[a,x]必须与[x,b]的矩阵相乘，否则就不能相乘，

In [31]:
a = torch.randn(6, 3)
b = torch.randn(3, 4)
c = torch.matmul(a, b)
c  # 这是一个 6*4 的矩阵，矩阵乘法是a的每一行，与 b 的每一列做点积运算，得到 c 的每一个元素
# 点积是两个向量对应元素相乘再求和，矩阵乘法是多个点积组成的，这就是点积与矩阵乘法的关系

## 向量的点积（点乘）
# torch.dot(x,y)跟 x @ y 是一样的，
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([4.0, 5.0, 6.0])
dot_product1 = torch.dot(a, b)  # 使用 torch.dot 计算点积
dot_product2 = a @ b  # 使用 @ 运算符计算点积
dot_product1, dot_product2  # 两种方法的结果是一样的

(tensor(32.), tensor(32.))

In [35]:
a,a/2,a.sum()

(tensor([1., 2., 3.]), tensor([0.5000, 1.0000, 1.5000]), tensor(6.))

In [39]:
q2 = torch.tensor([0.4, 1.4])
k1 = torch.tensor([0.3, 0.7])
k2 = torch.tensor([0.3, 1.1])
k3 = torch.tensor([0.3, 0.9])

d_model = 2

atten_scores1 = torch.dot(q2, k1) / (2**0.5)
atten_scores2 = torch.dot(q2, k2) / 2**0.5  
atten_scores3 = torch.dot(q2, k3) / 2**0.5

atten_scores1, atten_scores2, atten_scores3


                  

(tensor(0.7778), tensor(1.1738), tensor(0.9758))

In [3]:
import torch

test_data = torch.randn(4,4)

mask = torch.tril(torch.ones(6, 6), diagonal=1)

mask = torch.tril(torch.ones(6, 6))


print(mask)
a = mask[:4,:4]
print(a)
test_data1 = test_data.masked_fill(a == 0, float('-inf')) 
print(test_data1)

print(test_data)

mask2 = torch.triu(torch.ones(6, 6), diagonal=1)
print(mask2)
#        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))


b = mask2.bool()[:4,:4]
print(b)

test_data2 = test_data.masked_fill(b, -torch.inf)
print(test_data2)




tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[ 1.3002,    -inf,    -inf,    -inf],
        [ 0.1231, -0.7601,    -inf,    -inf],
        [ 0.4894,  0.2607, -2.2401,    -inf],
        [-0.6997,  1.1106,  1.5130,  0.3659]])
tensor([[ 1.3002, -0.8508, -1.1510, -2.3813],
        [ 0.1231, -0.7601,  0.0851, -2.5179],
        [ 0.4894,  0.2607, -2.2401,  1.5262],
        [-0.6997,  1.1106,  1.5130,  0.3659]])
tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
 

In [ ]:
for a in range(2, 5):
    print(a)